3D U-net Segmentation

In [0]:
import os
from tensorflow.keras.utils import get_file
import tensorflow as tf # version 2.2.0-rc2

import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import Adam, SGD
from tqdm import tqdm
import random
!pip install pyyaml h5py  # Required to save models in HDF5 format

cwd = os.getcwd()


Using TensorFlow backend.


Load MRI Data

In [0]:
# Create folders and define paths

!mkdir data
!mkdir results
!mkdir saved_model

DATA_PATH = '/content/datasets-promise12'
RESULT_PATH = '/content/results'

print('Image and label data downloaded: ')
print('Result directory created: <%s>.' % os.path.abspath(RESULT_PATH))

Image and label data downloaded: 
Result directory created: </content/results>.


In [0]:
import zipfile
!wget https://github.com/gu98/MPHY0041_Segmentation/blob/master/data/datasets-promise12.zip?raw=true
!unzip datasets-promise12.zip?raw=true

--2020-05-03 14:59:02--  https://github.com/gu98/MPHY0041_Segmentation/blob/master/data/datasets-promise12.zip?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/gu98/MPHY0041_Segmentation/raw/master/data/datasets-promise12.zip [following]
--2020-05-03 14:59:02--  https://github.com/gu98/MPHY0041_Segmentation/raw/master/data/datasets-promise12.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gu98/MPHY0041_Segmentation/master/data/datasets-promise12.zip [following]
--2020-05-03 14:59:02--  https://raw.githubusercontent.com/gu98/MPHY0041_Segmentation/master/data/datasets-promise12.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.co

Set Training Parameters

In [0]:
# Hyperparameters
path_to_data = 'DATA_PATH'
path_to_save = 'RESULT_PATH'

N= 50
N_test= 30

img_width= 128
img_height= 128
img_thickness= 32
img_channels= 1

learning_rate= float(5e-4)
epochs= 150
val_size= float(1/9)
test_size= 0.1
dropout= 0.5
batch_size= 4
patience= 150

#f= [0, 2, 4, 8, 16, 32, 16, 8, 4, 2]
#f= [0, 4, 8, 16, 32, 64, 32, 16, 8, 4]
#f= [0, 8, 16, 32, 64, 128, 64, 32, 16, 8]
num_channels = [0, 16, 32, 64, 128, 256, 128, 64, 32, 16]
f = num_channels
#f= [0, 32, 64, 128, 256, 512, 256, 128, 64, 32]
#f= [0, 64, 128, 256, 512, 1024, 512, 256, 128, 64]

#f= [0, 128, 256, 512, 1024, 2048, 1024, 512, 256, 128]


Parse Training Data

In [0]:
# Load Training Data
X_train = np.zeros((N, img_thickness, img_height, img_width, img_channels), dtype=np.float32)
Y_train = np.zeros((N, img_thickness, img_height, img_width, img_channels), dtype=np.float32)

print('','','')
print('','','')
print('Loading Training Data')

for n in tqdm(range(N)):
    image = np.load(os.path.join(DATA_PATH, "image_train%02d.npy" % n))
    label = np.load(os.path.join(DATA_PATH, "label_train%02d.npy" % n))
    X_train[n] = image[:,:,:,np.newaxis]
    Y_train[n] = label[:,:,:,np.newaxis]

print('Loaded Training Data')
tf.print(X_train.shape)
X_train = X_train[:int(X_train.shape[0]*(1-test_size))]
Y_train = Y_train[:int(Y_train.shape[0]*(1-test_size))]
tf.print(X_train.shape)

# Load Testing Data
X_test = np.zeros((N_test, img_thickness, img_height, img_width, img_channels), dtype=np.float32)

print('','','')
print('','','')
print('Loading Testing Data')

for n in tqdm(range(N_test)):
    image = np.load(os.path.join(DATA_PATH, "image_test%02d.npy" % n))
    X_test[n] = image[:,:,:,np.newaxis]

print('Loaded Testing Data')
print('','','')
print('','','')

100%|██████████| 50/50 [00:00<00:00, 664.46it/s]

  
  
Loading Training Data
Loaded Training Data
(50, 32, 128, 128, 1)
(45, 32, 128, 128, 1)



100%|██████████| 30/30 [00:00<00:00, 1098.44it/s]

  
  
Loading Testing Data
Loaded Testing Data
  
  


Metrics

In [0]:
from __future__ import division, print_function

from keras import backend as K
K.set_image_data_format('channels_last')


def dice_coef(y_true, y_pred, smooth=1.0):

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (
        K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

U-Net Model

In [0]:
# UNet Model
inputs = tf.keras.layers.Input((img_thickness, img_width, img_height, img_channels))

# Convert integers in image matrix to floating point 
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

# Encoding
c1 = tf.keras.layers.Conv3D(f[1], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.BatchNormalization(
     axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(c1)
c1 = tf.keras.layers.Dropout(dropout)(c1)
c2 = tf.keras.layers.Conv3D(f[1], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling3D((2,2,2))(c1)


c2 = tf.keras.layers.Conv3D(f[2], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.BatchNormalization(
     axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(c2)
c2 = tf.keras.layers.Dropout(dropout)(c2)
c2 = tf.keras.layers.Conv3D(f[2], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling3D((2,2,2))(c2)


c3 = tf.keras.layers.Conv3D(f[3], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.BatchNormalization(
     axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(c3)
c3 = tf.keras.layers.Dropout(dropout)(c3)
c3 = tf.keras.layers.Conv3D(f[3], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling3D((2,2,2))(c3)


c4 = tf.keras.layers.Conv3D(f[4], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.BatchNormalization(
     axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(c4)
c4 = tf.keras.layers.Dropout(dropout)(c4)
c4 = tf.keras.layers.Conv3D(f[4], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling3D((2,2,2))(c4)


c5 = tf.keras.layers.Conv3D(f[5], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.BatchNormalization(
     axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(c5)
c5 = tf.keras.layers.Dropout(dropout)(c5)
c5 = tf.keras.layers.Conv3D(f[5], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(c5)
p5 = tf.keras.layers.MaxPooling3D((2,2,2))(c5)

# Decoding Layers
u6 = tf.keras.layers.Conv3DTranspose(f[6], (2, 2, 2), strides=(2, 2, 2), padding='same',)(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv3D(f[6], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.BatchNormalization(
     axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(c6)
c6 = tf.keras.layers.Dropout(dropout)(c6)
c6 = tf.keras.layers.Conv3D(f[6], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(c6)


u7 = tf.keras.layers.Conv3DTranspose(f[7], (2, 2, 2), strides=(2, 2, 2), padding='same',)(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv3D(f[7], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.BatchNormalization(
     axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(c7)
c7 = tf.keras.layers.Dropout(dropout)(c7)
c7 = tf.keras.layers.Conv3D(f[7], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(c7)


u8 = tf.keras.layers.Conv3DTranspose(f[8], (2, 2, 2), strides=(2, 2, 2), padding='same',)(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv3D(f[8], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.BatchNormalization(
     axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(c8)
c8 = tf.keras.layers.Dropout(dropout)(c8)
c8 = tf.keras.layers.Conv3D(f[8], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(c8)


u9 = tf.keras.layers.Conv3DTranspose(f[9], (2, 2, 2), strides=(2, 2, 2), padding='same',)(c8)
u9 = tf.keras.layers.concatenate([u9, c1])
c9 = tf.keras.layers.Conv3D(f[9], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.BatchNormalization(
     axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(c9)
c9 = tf.keras.layers.Dropout(dropout)(c9)
c9 = tf.keras.layers.Conv3D(f[9], (3, 3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same')(c9)


outputs = tf.keras.layers.Conv3D(1, (1, 1, 1), activation='sigmoid')(c9)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer=tf.optimizers.Adam(learning_rate), loss=dice_coef_loss, metrics=[dice_coef])
model.summary()

callbacks = [tf.keras.callbacks.ModelCheckpoint('saved_model/best_model.h5',
                                                 verbose=1, save_best_only=True),
            tf.keras.callbacks.EarlyStopping(patience=patience, monitor='loss'),
            tf.keras.callbacks.TensorBoard(log_dir='logs')]

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 128, 128 0                                            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 32, 128, 128, 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv3d_76 (Conv3D)              (None, 32, 128, 128, 448         lambda_4[0][0]                   
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 32, 128, 128, 64          conv3d_76[0][0]                  
____________________________________________________________________________________________

Train the 3D U-Net

In [0]:
results = model.fit(X_train, Y_train, validation_split=val_size, batch_size=batch_size,
                    epochs=epochs, callbacks=callbacks) 
model.save('saved_model/final_model.h5') 

Epoch 1/150
10/10 [==============================] - ETA: 0s - loss: -0.0732 - dice_coef: 0.0732
Epoch 00001: val_loss improved from inf to -0.04834, saving model to saved_model/best_model.h5
10/10 [==============================] - 7s 682ms/step - loss: -0.0732 - dice_coef: 0.0732 - val_loss: -0.0483 - val_dice_coef: 0.0483
Epoch 2/150
10/10 [==============================] - ETA: 0s - loss: -0.1006 - dice_coef: 0.1006
Epoch 00002: val_loss did not improve from -0.04834
10/10 [==============================] - 6s 601ms/step - loss: -0.1006 - dice_coef: 0.1006 - val_loss: -0.0456 - val_dice_coef: 0.0456
Epoch 3/150
10/10 [==============================] - ETA: 0s - loss: -0.1407 - dice_coef: 0.1407
Epoch 00003: val_loss improved from -0.04834 to -0.05020, saving model to saved_model/best_model.h5
10/10 [==============================] - 6s 627ms/step - loss: -0.1407 - dice_coef: 0.1407 - val_loss: -0.0502 - val_dice_coef: 0.0502
Epoch 4/150
10/10 [==============================] - ETA:

Download Files

In [0]:
!zip -r ./logs.zip ./logs/
!zip -r ./saved_model ./saved_model/

  adding: logs/ (stored 0%)
  adding: logs/validation/ (stored 0%)
  adding: logs/validation/events.out.tfevents.1588519179.8ce87b3a89ad.123.56650.v2 (deflated 67%)
  adding: logs/validation/events.out.tfevents.1588518812.8ce87b3a89ad.123.34333.v2 (deflated 65%)
  adding: logs/validation/events.out.tfevents.1588519723.8ce87b3a89ad.123.81013.v2 (deflated 68%)
  adding: logs/validation/events.out.tfevents.1588518088.8ce87b3a89ad.123.7063.v2 (deflated 68%)
  adding: logs/train/ (stored 0%)
  adding: logs/train/events.out.tfevents.1588519130.8ce87b3a89ad.123.44933.v2 (deflated 93%)
  adding: logs/train/events.out.tfevents.1588518083.8ce87b3a89ad.profile-empty (deflated 5%)
  adding: logs/train/events.out.tfevents.1588518070.8ce87b3a89ad.123.1735.v2 (deflated 89%)
  adding: logs/train/events.out.tfevents.1588518802.8ce87b3a89ad.123.28988.v2 (deflated 92%)
  adding: logs/train/plugins/ (stored 0%)
  adding: logs/train/plugins/profile/ (stored 0%)
  adding: logs/train/plugins/profile/2020_05_

In [0]:
from google.colab import drive
drive.mount('/content/drive')